# 凯撒与哑谜

凯撒密码和哑谜都是文本加密的方法。这类方法还有很多。
为了更好的操作他们（以及破解），我们需要一套统一的接口：加密器类！
对应的，解密器也能被统一的组织起来：解密器类。

## 配置类 `Config`

在设计加密器和解密器类之前，可以发现他们都需要操作的一类重要数据即配置。
我们需要先定义一个配置类。
下面首先实现一个简单的能够容纳配置信息的类。

In [1]:
class Config:
    keys = []

    def __init__(self, **kwargs):
        self._config = {key: kwargs.get(key, None) for key in self.keys}

    def __getattr__(self, name):
        return self._config[name]

    def __str__(self):
        return str(self._config)

    def as_dict(self):
        return self._config


接下来实现一个凯撒密码的配置类，他只包含一个配置项，既位移`offset`。

In [2]:
class CaesarConfig(Config):
    keys = ["offset"]


caesar_config = CaesarConfig(offset=3)
print(caesar_config)


{'offset': 3}


哑谜配置则复杂一些，包括一组转子`rotors`，一个反射器`reflector`，一套插线板`plugboard`（上次未使用），以及一个转子初始位置`display`（上次明文给出为`ZEN`）。

In [3]:
class EnigmaConfig(Config):
    keys = ["rotors", "reflector", "plugboard", "display"]


enigma_config = EnigmaConfig(
    rotors=["I", "IV", "II"],
    reflector="B",
    plugboard="AB EZ CX DP KN TY",
    display="ZEN",
)
print(enigma_config)

{'rotors': ['I', 'IV', 'II'], 'reflector': 'B', 'plugboard': 'AB EZ CX DP KN TY', 'display': 'ZEN'}


## 加密器类 `Cipher`

接下来实现加密器类。注意配合`Config`类的使用。

In [4]:
class Cipher:
    def __init__(self, config):
        self.config = config

    def encrypt(self, text):
        raise NotImplementedError

    def decrypt(self, text):
        raise NotImplementedError

    def reset(self):
        pass  # default no-op

## 凯撒加密器

In [5]:
class CaesarCipher(Cipher):
    def rot(self, c, offset=None):
        offset = offset or self.config.offset
        if "A" <= c and c <= "Z":
            return chr(((ord(c) - 65) + offset) % 26 + 65)
        elif "a" <= c and c <= "z":
            return chr(((ord(c) - 97) + offset) % 26 + 97)
        return c

    def encrypt(self, text):
        return "".join(map(self.rot, text))

    def decrypt(self, text):
        return "".join(map(lambda c: self.rot(c, -self.config.offset), text))


In [6]:
# caesar_config = CaesarConfig(offset=3)
caesar_cipher = CaesarCipher(caesar_config)
print(caesar_cipher.encrypt("Python"))
print(caesar_cipher.decrypt("Sbwkrq"))


Sbwkrq
Python


### 哑谜加密器

In [7]:
# install py-engima
!pip install py-enigma

In [8]:
from enigma.machine import EnigmaMachine


class EnigmaCipher(Cipher):
    def __init__(self, config):
        super().__init__(config)
        print(self.config)
        self.machine = EnigmaMachine.from_key_sheet(
            rotors=self.config.rotors,
            reflector=self.config.reflector,
            plugboard_settings=self.config.plugboard
        )
        self.reset()

    def encrypt(self, text):
        return self.machine.process_text(text)

    decrypt = encrypt

    def reset(self):
        self.machine.set_display(self.config.display)


In [9]:
# enigma_config = EnigmaConfig(...)

enigma_cipher = EnigmaCipher(enigma_config)
print(enigma_cipher.encrypt("Python"))
print(enigma_cipher.decrypt("VIXQTI"))
enigma_cipher.reset()
print(enigma_cipher.decrypt("VIXQTI"))

{'rotors': ['I', 'IV', 'II'], 'reflector': 'B', 'plugboard': 'AB EZ CX DP KN TY', 'display': 'ZEN'}
VIXQTI
GUFUVB
PYTHON


## 破解器类 `Cracker`

In [ ]:
def gen_plugboard_v1(num_plugs=range(10), keys="ABCDEFGHIJKLMNOPQRSTUVWXYZ"):
    # this generates dummy settings for the plugboard, students can try to 
    def gen_plugs():
        for plug in combinations(keys, 2):
            yield "".join(plug)
    for plugs in num_plugs:
        for plugboard in product(gen_plugs(), repeat=plugs):
            yield plugboard

def gen_plugboard_v2(num_plugs=range(10), keys="ABCDEFGHIJKLMNOPQRSTUVWXYZ"):
    # this generates dummy settings for the plugboard, students can try to 
    def gen_plugs(num_plugs, keys):
        if num_plugs == 0:
            yield ""
        else:
            for plug in combinations(keys, 2):
                reduced_keys = keys.replace(plug[0], "").replace(plug[1], "")
                for sub_plug in gen_plugs(num_plugs-1, reduced_keys):
                    yield ("".join(plug) + " " + sub_plug).strip()
    for plugs in num_plugs:
        yield from gen_plugs(plugs, keys)

In [4]:
from itertools import permutations

ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"


def plugboard_gen():
    for p in permutations(ALPHABET, r=12):
        yield " ".join(map(lambda t: t[0] + t[1], zip(p[::2], p[1::2])))

In [ ]:
def pb_loopy():
    visited = set()
    for a in "ABCDEF":
        for b in "ABCDEF":
            for c in "ABCDEF":
                for d in "ABCDEF":
                    if a + b + " " + c + d in:
                        continue
                    yield a + b + " " + c + d

In [5]:
def plugboard_gen_r(alphabet, lines):
    if len(alphabet) < lines * 2 or lines == 0:
        return  # stop iteration
    
    a, ralphabet = alphabet[0], alphabet[1:]
    # with a
    if lines == 1:
        for b in ralphabet:
            yield f"{a}{b}"
    else:
        for i, b in enumerate(ralphabet):
            left = ralphabet[:i]
            right = ralphabet[i+1:]
            for tail in plugboard_gen_r(left + right, lines-1):
                yield f"{a}{b} {tail}"
        
    # without a
    yield from plugboard_gen_r(ralphabet, lines)

for p in plugboard_gen_r(list("abcdefgh"), lines=3):
    print(p)

ab cd ef
ab cd eg
ab cd eh
ab cd fg
ab cd fh
ab cd gh
ab ce df
ab ce dg
ab ce dh
ab ce fg
ab ce fh
ab ce gh
ab cf de
ab cf dg
ab cf dh
ab cf eg
ab cf eh
ab cf gh
ab cg de
ab cg df
ab cg dh
ab cg ef
ab cg eh
ab cg fh
ab ch de
ab ch df
ab ch dg
ab ch ef
ab ch eg
ab ch fg
ab de fg
ab de fh
ab de gh
ab df eg
ab df eh
ab df gh
ab dg ef
ab dg eh
ab dg fh
ab dh ef
ab dh eg
ab dh fg
ab ef gh
ab eg fh
ab eh fg
ac bd ef
ac bd eg
ac bd eh
ac bd fg
ac bd fh
ac bd gh
ac be df
ac be dg
ac be dh
ac be fg
ac be fh
ac be gh
ac bf de
ac bf dg
ac bf dh
ac bf eg
ac bf eh
ac bf gh
ac bg de
ac bg df
ac bg dh
ac bg ef
ac bg eh
ac bg fh
ac bh de
ac bh df
ac bh dg
ac bh ef
ac bh eg
ac bh fg
ac de fg
ac de fh
ac de gh
ac df eg
ac df eh
ac df gh
ac dg ef
ac dg eh
ac dg fh
ac dh ef
ac dh eg
ac dh fg
ac ef gh
ac eg fh
ac eh fg
ad bc ef
ad bc eg
ad bc eh
ad bc fg
ad bc fh
ad bc gh
ad be cf
ad be cg
ad be ch
ad be fg
ad be fh
ad be gh
ad bf ce
ad bf cg
ad bf ch
ad bf eg
ad bf eh
ad bf gh
ad bg ce
ad bg cf
ad bg ch
a

In [ ]:
def plugboard_gen_r(alphabet, lines):
    if len(alphabet) < lines * 2 or lines == 0:
        return  # stop iteration
    
    a, ralphabet = alphabet[0], alphabet[1:]
    # with a
    if lines == 1:
        for b in ralphabet:
            yield f"{a}{b}"
    else:
        for i, b in enumerate(ralphabet):
            left = ralphabet[:i]
            right = ralphabet[i+1:]
            for tail in plugboard_gen_r(left + right, lines-1):
                yield f"{a}{b} {tail}"
        
    # without a
    yield from plugboard_gen_r(ralphabet, lines)

for p in plugboard_gen_r(list("abcdefgh"), lines=3):
    print(p)

In [7]:
from itertools import count
from tqdm.auto import tqdm


def plugboard_gen():
    yield from plugboard_gen_r(list(ALPHABET), lines=6)

for i, _ in enumerate(tqdm(plugboard_gen())):
    pass


print(i)


0it [00:00, ?it/s]

100391791499


In [17]:
from itertools import product
from enigma.rotors.data import ROTORS, REFLECTORS


class EnigmaCracker:
    def crack(self, config, text, crib):
        if config.rotors is None:
            rotors = permutations(ROTORS.keys(), r=3)
        else:
            rotors = [config.rotors]
        if config.reflector is None:
            reflectors = REFLECTORS.keys()
        else:
            reflectors = [config.reflector]
        if config.plugboard is None:
            plugboards = plugboard_gen()
        else:
            plugboards = [config.plugboard]
        if config.display is None:
            displays = permutations(ALPHABET, r=3)
        else:
            displays = [config.display]
        for rotor, reflector, plugboard, display in product(rotors, reflectors, plugboards, displays):
            config = {
                "rotors": rotor,  # 3个转子，如 IV II III
                "reflector": reflector,  # 1个反射器，如 B
                "plugboard_settings": plugboard,
            }
            machine = EnigmaMachine.from_key_sheet(**config)
            machine.set_display(display)
            recovered = machine.process_text(text)
            if crib.upper() in recovered:
                print(config)
                break
        else:
            print("Not found.")


In [18]:

enigma_tamplate = EnigmaConfig(
    plugboard="AB EZ CX DP KN TY",
    display="ZEN",
)

EnigmaCracker().crack(enigma_tamplate, "VIXQTI", "PYTHON")

{'rotors': ('I', 'IV', 'II'), 'reflector': 'B', 'plugboard_settings': 'AB EZ CX DP KN TY'}


## That's all folks!

### 自动配置器

为了辅助和简化破解，我们可以设计一个自动配置器，提供每个配置项的允许值。

In [ ]:
from itertools import product


class AutoConfig(Config):
    options = {}

    def options_gen(self, key):
        if key in self._config and self._config[key] is not None:
            yield self._config[key]
        return self.options[key]
    
    def auto_gen(self):
        option_list = list(map(list, map(self.options_gen, self.keys)))
        print(option_list)
        for option in product(*option_list):
            print(option)
            yield type(self)(**{key: value for key, value in zip(self.keys, option)})


In [ ]:
class AutoCaesarConfig(AutoConfig, CaesarConfig):
    options = {"offset": range(26),}

In [ ]:
from enigma.rotors.data import ROTORS, REFLECTORS


class AutoEnigmaConfig(AutoConfig, EnigmaConfig):
    options = {
        "rotors": permutations(ROTORS.keys(), r=3),
        "reflector": REFLECTORS,
        "plugboard": plugboard_gen(),
        "display": permutations(ALPHABET, r=3),
    }
